In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import subprocess
import os
# python version: 3.11.9

In [2]:
# presets, functions

# preset values for the analysis:
#____________________________________________________________________________________________________________________
# physical constants
Z = 1
A = 2
mass_nucleon = 0.938273 
# mass_nucleus = 11.178
mass_nucleus = A * 0.931494
alpha_fine = 1 / 137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']

def round_sig_3(x):
    return '%s' % float('%.3g' % x)

def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def linear_model(x, a, b):
    return a * x + b

# read dataframe from csv file
def prepare_df(filepath):
    # read the csv file:
    df = pd.read_csv(filepath)

    df["Veff"] = 0.0005
    
    df["Eeff"] = df["E0"] + df["Veff"]
    df["Ep"] = df["E0"] - df["nu"]
    df["Ep_eff"] = df["Ep"] + df["Veff"]

    # calculate normalized cross section:
    df['normalization'] = 1.0
    df['normError'] = 0.0
    df['system_err'] = 0.0
    df['normCross'] = df['cross'] * df['normalization']
    df['normCrossError'] = (np.sqrt(df['error']**2 + ((df['system_err'] * df['cross'])**2))) * df['normalization']
    print(df.loc[df['normalization'] == 1, 'dataSet'].unique())
    
    # calculate the kinematic variables:
    df["ThetaRad"] = df["ThetaDeg"]*np.pi/180
    df["sin2(T/2)"] = (np.sin(df["ThetaRad"] / 2))**2
    df["cos2(T/2)"] = (np.cos(df["ThetaRad"] / 2))**2
    df["tan2(T/2)"] = (np.tan(df["ThetaRad"] / 2))**2

    df["nuel"] = df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus)
    df["Ex"] = df["nu"] - (df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus))

    df["R"] = 1.1 * (df["A"])**(1/3) + 0.86 / ((df["A"])**(1/3))

    df["F2foc"] = (df["Eeff"] / df["E0"])**2
    df["Q2"] = 4 * df["E0"] * (df["Ep"]) * df["sin2(T/2)"]
    df["Q2eff"] = 4 * df["Eeff"] * df["Ep_eff"] * df["sin2(T/2)"]
    df["qv2"] = df["nu"]**2 + df["Q2eff"]
    df["qv"] = np.sqrt(df["qv2"])

    df["W2"] = mass_nucleon**2 + 2 * mass_nucleon * df["nu"] - df["Q2eff"]
    df["epsilon"] = 1 / (1 + 2 * (1 + (df["nu"]**2) / df["Q2eff"]) * df["tan2(T/2)"])

    df["gamma"] = alpha_fine * df["Ep_eff"] * (df["W2"] - mass_nucleon**2) / (( 4 * ((np.pi)**2) * df["Q2eff"] * mass_nucleon * df["E0"]) * (1 - df["epsilon"]))
    df["Sig_R"] = df["normCross"] / df["gamma"]
    df["D_sig_R"] = df["error"] / df["gamma"]
    df["Sig_mott"] = 4 * (alpha_fine**2) * (df["Ep"]**2) * df["cos2(T/2)"] / (df["Q2"]**2)
    df["Sig_mott_eff"] = df["Sig_mott"] * df["E0"] / df["Eeff"]

    # Calculate the Rosenbluth quantity:
    df["H"] = (df["qv2"]**2) / (4 * (alpha_fine**2) * (df["Ep_eff"]**2) * (df["cos2(T/2)"] + 2 * (df["qv2"] / df["Q2eff"]) * df["sin2(T/2)"]))
    df["Hcc"] = df["H"] / df["F2foc"]
    df["Hstar_Sig(nb)"] = df["H"] * df["normCross"]
    df["Hstar_error(nb)"] = df["H"] * df["normCrossError"]
    df["Hstar_Sig(GeV)"] = df["Hstar_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hstar_error(GeV)"] = df["Hstar_error(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_Sig(nb)"] = df["Hcc"] * df["normCross"]
    df["Hcc_error(nb)"] = df["Hcc"] * df["normCrossError"]
    df["Hcc_Sig(GeV)"] = df["Hcc_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_error(GeV)"] = df["Hcc_error(nb)"] / ((0.1973269**2) * 10000000)
    # we will fit "Hcc_Sig(GeV)" vs "epsilon" to the linear model to get the Rosenbluth slope and intercept

    # subdivide the data into bins
    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvbin_names, right=True)
    df["qvcenter"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvcenters, right=True)
    df['qvcenter'] = pd.to_numeric(df['qvcenter'])
    df["Q2bin"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2bin_names, right=True)
    df["Q2center"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2centers, right=True)
    df['Q2center'] = pd.to_numeric(df['Q2center'])
    df = df.dropna()
    # now every row of data has a bin (Q2/qv) label

    return df

In [3]:
# read csv file
df = prepare_df('Data/H2.csv')
df

[ 1  2  3  4  5  6  7  8 13 14 16]


,Z,A,E0,ThetaDeg,nu,cross,error,dataSet,Veff,Eeff,...,Hstar_Sig(GeV),Hstar_error(GeV),Hcc_Sig(nb),Hcc_error(nb),Hcc_Sig(GeV),Hcc_error(GeV),qvbin,qvcenter,Q2bin,Q2center
0,1,2,2.3528,43.3976,1.8168,108000.0,4010.0,1,0.0005,2.3533,...,29471.377806,1094.261343,1.147066e+10,4.259014e+08,29458.855711,1093.796402,"[1.770,2.067]",1.921,"[0.650,1.050]",0.800
1,1,2,2.3038,30.7459,1.8270,166000.0,5780.0,1,0.0005,2.3043,...,45134.796936,1571.561002,1.756692e+10,6.116674e+08,45115.211855,1570.879063,"[1.770,2.067]",1.921,"[0.206,0.322]",0.265
2,1,2,2.3593,43.4006,1.7727,107000.0,4020.0,1,0.0005,2.3598,...,25057.611800,941.416817,9.752775e+09,3.664127e+08,25046.994393,941.017920,"[1.770,2.067]",1.921,"[0.650,1.050]",0.800
3,1,2,2.3682,43.3960,1.7456,103000.0,3820.0,1,0.0005,2.3687,...,21893.973394,811.990081,8.521456e+09,3.160385e+08,21884.731336,811.647318,"[1.770,2.067]",1.921,"[0.650,1.050]",0.800
4,1,2,2.3040,30.7485,1.7479,160000.0,5510.0,1,0.0005,2.3045,...,32113.256864,1105.900283,1.249880e+10,4.304276e+08,32099.323354,1105.420448,"[1.770,2.067]",1.921,"[0.322,0.438]",0.380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543,1,2,7.0000,10.0000,2.7100,240.0,14.1,16,0.0005,7.0005,...,9.479755,0.556936,3.690691e+06,2.168281e+05,9.478401,0.556856,"[2.642,2.923]",2.783,"[0.650,1.050]",0.800
6544,1,2,7.0000,10.0000,2.6600,248.0,16.9,16,0.0005,7.0005,...,9.029419,0.615311,3.515364e+06,2.395551e+05,9.028129,0.615223,"[2.642,2.923]",2.783,"[0.650,1.050]",0.800
6545,1,2,7.0000,10.0000,2.6100,259.0,17.9,16,0.0005,7.0005,...,8.686236,0.600323,3.381755e+06,2.337198e+05,8.684996,0.600237,"[2.642,2.923]",2.783,"[0.650,1.050]",0.800
6546,1,2,7.0000,10.0000,2.5600,263.0,15.8,16,0.0005,7.0005,...,8.119094,0.487763,3.160953e+06,1.898976e+05,8.117934,0.487693,"[2.642,2.923]",2.783,"[0.650,1.050]",0.800


In [4]:
response_columns = ['i','RTTOT','RLTOT','RTQE','RLQE','RTNS','RLNS']
C12_factor = Z / 6

# calculate bc_qv_nu
df[['qvcenter','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_qvc_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) # qvc: qv at bin center
df['RT_qvc_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) # qvd: qv of data

df[['qv','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_qvd_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_nu done.')

# calculate bc_qv_ex
df[['qvcenter','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_qvc_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_qvd_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_ex done.')

# calculate bc_qv_w2
df[['qvcenter','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_qvc_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_qvd_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_w2 done.')

# calculate bc_q2_nu
df[['Q2center','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_q2c_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_q2d_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_nu done.')

# calculate bc_q2_ex
df[['Q2center','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_q2c_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_q2d_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_ex done.')

# calculate bc_q2_w2
df[['Q2center','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor
df['RL_q2c_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df_response['RLTOT'] *= C12_factor
df_response['RTTOT'] *= C12_factor 
df['RL_q2d_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_w2 done.')


<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:41: SyntaxWarning: invalid escape sequence '\s'
<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:63: SyntaxWarning: invalid escape sequence '\s'
<>:75: SyntaxWarning: invalid escape sequence '\s'
<>:85: SyntaxWarning: invalid escape sequence '\s'
<>:97: SyntaxWarning: invalid escape sequence '\s'
<>:107: SyntaxWarning: invalid escape sequence '\s'
<>:119: SyntaxWarning: invalid escape sequence '\s'
<>:129: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:41: SyntaxWarning: invalid escape sequence '\s'
<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:63: SyntaxWarning: invalid escape sequence '\s'
<>:75: SyntaxWarning: invalid escape sequence '\s'
<>:85: SyntaxWarning: invalid 

RL RT qv_nu done.
RL RT qv_ex done.
RL RT qv_w2 done.
RL RT q2_nu done.
RL RT q2_ex done.
RL RT q2_w2 done.


In [5]:
# determine bin-centering corrections
df["bc_qv_nu"] = 1.0
df["bc_qv_ex"] = 1.0
df["bc_qv_w2"] = 1.0
df["bc_q2_nu"] = 1.0
df["bc_q2_ex"] = 1.0
df["bc_q2_w2"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]
    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN') and row['Ex'] >= 0.025:
            try:
                df.loc[index, 'bc_qv_nu'] = (row['epsilon'] * row['RL_qvc_nu'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu']**2)) * row['RT_qvc_nu']) / (row['epsilon'] * row['RL_qvd_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_ex'] = (row['epsilon'] * row['RL_qvc_ex'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu']**2)) * row['RT_qvc_ex']) / (row['epsilon'] * row['RL_qvd_ex'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_w2 due to zero denominator.')

for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]
    for index, row in picked.iterrows():
        if row['Q2center'] != float('NaN') and row['Ex'] >= 0.025:
            nu = (row['W2'] + Q2center - mass_nucleon**2) / (2 * mass_nucleon)
            qv2center = Q2center + nu**2
            try:
                df.loc[index, 'bc_q2_nu'] = (row['epsilon'] * row['RL_q2c_nu'] + 0.5 * ((Q2center + row["nu"]**2) / (Q2center)) * row['RT_q2c_nu']) / (row['epsilon'] * row['RL_q2d_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_ex'] = (row['epsilon'] * row['RL_q2c_ex'] + 0.5 * ((Q2center + row["nu"]**2) / (Q2center)) * row['RT_q2c_ex']) / (row['epsilon'] * row['RL_q2d_ex'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_w2'] = (row['epsilon'] * row['RL_q2c_w2'] + 0.5 * (qv2center / (Q2center)) * row['RT_q2c_w2']) / (row['epsilon'] * row['RL_q2d_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_w2 due to zero denominator.')

Skipping index 6102 for bc_qv_nu due to zero denominator.
Skipping index 6102 for bc_qv_ex due to zero denominator.
Skipping index 6527 for bc_qv_nu due to zero denominator.
Skipping index 6527 for bc_qv_ex due to zero denominator.


In [6]:
# save dataframe to csv
df.to_csv('Data/df_H2.csv',index=False)